In [18]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [19]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.1_wasting_treatment/ciff_sam/2021_09_24_16_36_30/count_data/'

In [20]:
state_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv')
state_pt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age
0,0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,early_neonatal
1,1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,early_neonatal
2,2,female,2022,mild_child_wasting,state_person_time,29,baseline,201.939767,uncovered,covered,early_neonatal
3,3,female,2022,mild_child_wasting,state_person_time,29,baseline,207.442847,uncovered,uncovered,early_neonatal
4,4,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,late_neonatal


In [21]:
state_pt.year.unique()

array([2022, 2023, 2024, 2025, 2026])

In [22]:
state_pt.age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [30]:
groupby_cols = ['scenario','input_draw','cause']

def calculate_child_growth_exposure_prevalence(risk):
    state_pt = pd.read_csv(output_dir +f'{risk}_state_person_time.csv')
    state_pt = state_pt.loc[state_pt.year>=2023].loc[state_pt.age.isin(['6-11_months',
       '12_to_23_months', '2_to_4'])]
    #state_pt = state_pt.loc[state_pt.scenario=='baseline']
    state_pt = state_pt.groupby(groupby_cols).sum()[['value']].reset_index()
    if risk=='wasting':
        state_pt_exposed = state_pt.loc[state_pt.cause.str.contains('acute_malnutrition')]
        state_pt_exposed['cause'] = 'wasted'
    else:
        state_pt_exposed = state_pt.loc[state_pt.cause.isin(['cat1','cat2'])]
        state_pt_exposed['cause'] = 'stunted'
    state_pt_with_exposed = (pd.concat([state_pt, 
                          state_pt_exposed.groupby(groupby_cols).sum().reset_index()],
                         ignore_index=True)
                .set_index(groupby_cols))
    prev = state_pt_with_exposed / state_pt.groupby([c for c in groupby_cols if c != 'cause']).sum()
    prev = prev.groupby([c for c in groupby_cols if c != 'input_draw']).describe(percentiles=[0.025,0.975]).sort_index()
    return prev

In [31]:
test = calculate_child_growth_exposure_prevalence('wasting')
test

/ihme/code/alibow/miniconda/envs/vivarium_ciff_sam/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


value                      \
                                               count      mean       std   
scenario          cause                                                    
baseline          mild_child_wasting            12.0  0.216405  0.001265   
                  moderate_acute_malnutrition   12.0  0.078580  0.000836   
                  severe_acute_malnutrition     12.0  0.017591  0.000987   
                  susceptible_to_child_wasting  12.0  0.687424  0.001548   
                  wasted                        12.0  0.096171  0.001759   
sqlns             mild_child_wasting            12.0  0.233159  0.002107   
                  moderate_acute_malnutrition   12.0  0.058470  0.001786   
                  severe_acute_malnutrition     12.0  0.012832  0.000482   
                  susceptible_to_child_wasting  12.0  0.695540  0.001239   
                  wasted                        12.0  0.071301  0.002073   
wasting_treatment mild_child_wasting            12.0  0.226342  0.001779   
                  moderate_acute_malnutrition   12.0  0.066912  0.001930   
                  severe_acute_malnutrition     12.0  0.014405  0.000674   
                  susceptible_to_child_wasting  12.0  0.692341  0.001621   
                  wasted                        12.0  0.081317  0.002447   

                                                                              \
                                                     min      2.5%       50%   
scenario          cause                                                        
baseline          mild_child_wasting            0.213816  0.214125  0.216489   
                  moderate_acute_malnutrition   0.077275  0.077334  0.078499   
                  severe_acute_malnutrition     0.016161  0.016326  0.017326   
                  susceptible_to_child_wasting  0.685123  0.685293  0.687037   
                  wasted                        0.093436  0.093739  0.095739   
sqlns             mild_child_wasting            0.229849  0.229951  0.233002   
                  moderate_acute_malnutrition   0.055080  0.055463  0.058559   
                  severe_acute_malnutrition     0.012034  0.012127  0.012760   
                  susceptible_to_child_wasting  0.693161  0.693525  0.695556   
                  wasted                        0.067114  0.067637  0.071575   
wasting_treatment mild_child_wasting            0.223740  0.224109  0.225771   
                  moderate_acute_malnutrition   0.063309  0.063560  0.067199   
                  severe_acute_malnutrition     0.013566  0.013618  0.014229   
                  susceptible_to_child_wasting  0.689446  0.689946  0.692001   
                  wasted                        0.076875  0.077240  0.081697   

                                                                    
                                                   97.5%       max  
scenario          cause                                             
baseline          mild_child_wasting            0.218231  0.218441  
                  moderate_acute_malnutrition   0.079860  0.079881  
                  severe_acute_malnutrition     0.019570  0.019934  
                  susceptible_to_child_wasting  0.690185  0.690521  
                  wasted                        0.099430  0.099816  
sqlns             mild_child_wasting            0.236195  0.236327  
                  moderate_acute_malnutrition   0.061013  0.061372  
                  severe_acute_malnutrition     0.013603  0.013618  
                  susceptible_to_child_wasting  0.697287  0.697297  
                  wasted                        0.073838  0.073995  
wasting_treatment mild_child_wasting            0.229789  0.229894  
                  moderate_acute_malnutrition   0.069802  0.070081  
                  severe_acute_malnutrition     0.015653  0.015862  
                  susceptible_to_child_wasting  0.695238  0.695604  
                  wasted                        0.085035  0.0850

In [29]:
test.reset_index().loc[test.reset_index().cause==['moderate_acute_malnutrition']]

ValueError: ('Lengths must match to compare', (15,), (1,))